In [1]:
import argparse
import numpy as np
import pandas as pd

In [2]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [3]:
from lib import utils
from lib.metrics import masked_rmse_np, masked_mape_np, masked_mae_np
from lib.utils import StandardScaler

In [4]:
log_dir = './'
log_level = 'INFO'
logger = utils.get_logger(log_dir, __name__, 'info.log', level=log_level)

2021-03-18 18:31:59,758 - INFO - Log directory: ./


### 
- pass the historical data as some training set
- let the periodicy and time period be understood

In [ ]:
def historical_average_new(train_df, )

In [5]:
def historical_average_predict(df, y_test, n_train, n_sample, period, null_val):
    
    y_predict = pd.DataFrame.copy(y_test)
    
    # performing test directly (from the end of training data to the very end)
    for i in range(n_train, min(n_sample, n_train + period)):
        #print(i)
        
        # from current to end of train with a periodicy of week
        inds = [j for j in range(i % period, n_train, period)]
        
        # all the instances where...
        historical = df.iloc[inds, :]
        
        # mean of past wednesdays
        y_predict.iloc[i - n_train, :] = historical[historical != null_val].mean()
        
    # Copy each period.
    for i in range(n_train + period, n_sample, period):
        size = min(period, n_sample - i)
        
        start = i - n_train
        
        y_predict.iloc[start:start + size, :] = y_predict.iloc[start - period: start + size - period, :].values
        
    return y_predict, y_test

### I  want:
- inputs, predictions, ground truths for train and test sets
- a way to test inputs on a trained traditional model

### how do I make this work with the PeMS data?
- which is either available as downloaded files from PeMS
- or the train test val npz files
- I need df here

In [6]:
# CONSTANTS
TEST_RATIO = 0.2
PERIOD = 7 * 24 * 12
NULL_VALUE = 0.

In [7]:
# The data is sensor_volume_150.csv
data = pd.read_csv('./data/sensor_volume_150.csv')

n_sample, n_sensor = data.shape
print(f"Total Samples: {n_sample}, Total Sensors: {n_sensor}")

n_test = int(round(n_sample * TEST_RATIO))
print(f"\nTest Samples: {n_test}")

n_train = n_sample - n_test
print(f"\nTrain Samples: {n_train}")

# Test set, 20% from behind
y_test = data[-n_test:]

Total Samples: 13104, Total Sensors: 150

Test Samples: 2621

Train Samples: 10483


In [8]:
# y_predict means predictions, y_test means ground truths
y_predict, y_test = historical_average_predict(data, y_test, n_train, n_sample, PERIOD, NULL_VALUE)

In [9]:
# print GCGRNN/ DCRNN metrics
rmse = masked_rmse_np(preds=y_predict.values, labels=y_test.values, null_val=0)
mape = masked_mape_np(preds=y_predict.values, labels=y_test.values, null_val=0)
mae = masked_mae_np(preds=y_predict.values, labels=y_test.values, null_val=0)

logger.info('Historical Average')
logger.info('\t'.join(['Model', 'Horizon', 'RMSE', 'MAPE', 'MAE']))
for horizon in range(12):
    line = 'HA\t%d\t%.2f\t%.2f\t%.2f' % (horizon, rmse, mape * 100, mae)
    logger.info(line)

2021-03-18 18:32:27,366 - INFO - Historical Average
2021-03-18 18:32:27,367 - INFO - Model	Horizon	RMSE	MAPE	MAE
2021-03-18 18:32:27,367 - INFO - HA	0	909.79	27.13	591.10
2021-03-18 18:32:27,368 - INFO - HA	1	909.79	27.13	591.10
2021-03-18 18:32:27,368 - INFO - HA	2	909.79	27.13	591.10
2021-03-18 18:32:27,369 - INFO - HA	3	909.79	27.13	591.10
2021-03-18 18:32:27,369 - INFO - HA	4	909.79	27.13	591.10
2021-03-18 18:32:27,369 - INFO - HA	5	909.79	27.13	591.10
2021-03-18 18:32:27,370 - INFO - HA	6	909.79	27.13	591.10
2021-03-18 18:32:27,370 - INFO - HA	7	909.79	27.13	591.10
2021-03-18 18:32:27,371 - INFO - HA	8	909.79	27.13	591.10
2021-03-18 18:32:27,371 - INFO - HA	9	909.79	27.13	591.10
2021-03-18 18:32:27,372 - INFO - HA	10	909.79	27.13	591.10
2021-03-18 18:32:27,372 - INFO - HA	11	909.79	27.13	591.10


In [10]:
def var_predict(df, n_forwards=(1, 3), n_lags=4, test_ratio=0.2):
    """
    Multivariate time series forecasting using Vector Auto-Regressive Model.
    :param df: pandas.DataFrame, index: time, columns: sensor id, content: data.
    :param n_forwards: a tuple of horizons.
    :param n_lags: the order of the VAR model.
    :param test_ratio:
    :return: [list of prediction in different horizon], dt_test
    """
    n_sample, n_output = df.shape
    n_test = int(round(n_sample * test_ratio))
    n_train = n_sample - n_test
    df_train, df_test = df[:n_train], df[n_train:]

    scaler = StandardScaler(mean=df_train.values.mean(), std=df_train.values.std())
    data = scaler.transform(df_train.values)
    var_model = VAR(data)
    var_result = var_model.fit(n_lags)
    max_n_forwards = np.max(n_forwards)
    # Do forecasting.
    result = np.zeros(shape=(len(n_forwards), n_test, n_output))
    start = n_train - n_lags - max_n_forwards + 1
    for input_ind in range(start, n_sample - n_lags):
        prediction = var_result.forecast(scaler.transform(df.values[input_ind: input_ind + n_lags]), max_n_forwards)
        for i, n_forward in enumerate(n_forwards):
            result_ind = input_ind - n_train + n_lags + n_forward - 1
            if 0 <= result_ind < n_test:
                result[i, result_ind, :] = prediction[n_forward - 1, :]

    df_predicts = []
    for i, n_forward in enumerate(n_forwards):
        df_predict = pd.DataFrame(scaler.inverse_transform(result[i]), index=df_test.index, columns=df_test.columns)
        df_predicts.append(df_predict)
    return df_predicts, df_test

def eval_var(traffic_reading_df, n_lags=3):
    n_forwards = [1, 3, 6, 12]
    y_predicts, y_test = var_predict(traffic_reading_df, n_forwards=n_forwards, n_lags=n_lags,
                                     test_ratio=0.2)
    logger.info('VAR (lag=%d)' % n_lags)
    logger.info('Model\tHorizon\tRMSE\tMAPE\tMAE')
    for i, horizon in enumerate(n_forwards):
        rmse = masked_rmse_np(preds=y_predicts[i].values, labels=y_test.values, null_val=0)
        mape = masked_mape_np(preds=y_predicts[i].values, labels=y_test.values, null_val=0)
        mae = masked_mae_np(preds=y_predicts[i].values, labels=y_test.values, null_val=0)
        line = 'VAR\t%d\t%.2f\t%.2f\t%.2f' % (horizon, rmse, mape * 100, mae)
        logger.info(line)

In [11]:
# VAR
eval_var(data, n_lags=3)

2021-03-18 18:32:53,221 - INFO - VAR (lag=3)
2021-03-18 18:32:53,222 - INFO - Model	Horizon	RMSE	MAPE	MAE
2021-03-18 18:32:53,251 - INFO - VAR	1	373.76	9.92	262.43
2021-03-18 18:32:53,280 - INFO - VAR	3	748.59	22.67	547.59
2021-03-18 18:32:53,314 - INFO - VAR	6	937.20	30.20	690.56
2021-03-18 18:32:53,350 - INFO - VAR	12	1044.11	32.45	771.60
